# word2vec 속도 개선
- 앞 장의 CBOW의 경우, 말뭉치에 포함된 어휘수가 많아지면 계산량도 커진다는 단점이 있다.
- Embedding과 negative sampling로 개선을 하여 속도를 향상 시킬 것이다.

## Embedding 계층


In [1]:
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
        
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
#         for i, word_id in enumerate(self.idx):
#             dW[word_id] = += dout[i]
        np.add.at(dW, self.idx, dout)
        return None

## 네거티브 샘플링
- 핵심 : 다중분류 -> 이진분류

In [3]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
        
    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W*h, axis=1)
        
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, taget_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)
        
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

In [7]:
from ch04.negative_sampling_layer import UnigramSampler, SigmoidWithLoss
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power =.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]
        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads
            
        def forward(self, h, target):
            batch_size = target.shape[0]
            negative_sample = self.sampler.get_negative_sample(target)
            
            score = self.embed_dot_layers[0].forward(h, target)
            correcct_label = np.ones(batch_size, dtype=np.int32)
            loss = self.loss_layers[0].forward(score, correcct_label)
            
            negative_label = np.zeros(batch_size, dtype=np.int32)
            for i in range(self.sample_size):
                negative_target = negative_sample[:, i]
                score = self.embed_dot_layers[1+i].forward(h, negative_target)
                loss += self.loss_layers[1+i].forward(score, negative_label)
            return loss
        
        def backward(self, dout = 1):
            dh = 0
            for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
                dscore = l0.backward(dout)
                dh += l1.backward(dscore)
            return dh

In [8]:
class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size
        
        W_in = .01 * np.random.randn(V, H).astype('f')
        W_out = .01 * np.random.randn(V, H).astype('f')
        
        self.in_layers=[]
        for i in range(2*window_size):
            layer = Embedding(W_in)
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=.75, sample_size=5)
        
        layers = self.in_layers +[self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
            
        self.word_vecs = W_in
        
    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h+= layer.forward(contexts[:,i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss
    
    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1/len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

In [ ]:
from common import config
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb
from ch04.negative_sampling_layer import NegativeSamplingLoss
from common.layers import Embedding

window_size = 5
hidden_size = 100
batch_size = 100
max_epochs = 10

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)
    
model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

trainer.fit(contexts, target, max_epochs, batch_size)
trainer.plot()

word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)

params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)

| 에폭 1 |  반복 1 / 9295 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 21 / 9295 | 시간 1[s] | 손실 4.16
| 에폭 1 |  반복 41 / 9295 | 시간 2[s] | 손실 4.15
| 에폭 1 |  반복 61 / 9295 | 시간 3[s] | 손실 4.12
| 에폭 1 |  반복 81 / 9295 | 시간 4[s] | 손실 4.05
| 에폭 1 |  반복 101 / 9295 | 시간 5[s] | 손실 3.93
| 에폭 1 |  반복 121 / 9295 | 시간 6[s] | 손실 3.79
| 에폭 1 |  반복 141 / 9295 | 시간 7[s] | 손실 3.64
| 에폭 1 |  반복 161 / 9295 | 시간 8[s] | 손실 3.48
| 에폭 1 |  반복 181 / 9295 | 시간 9[s] | 손실 3.37
| 에폭 1 |  반복 201 / 9295 | 시간 10[s] | 손실 3.26
| 에폭 1 |  반복 221 / 9295 | 시간 11[s] | 손실 3.17
| 에폭 1 |  반복 241 / 9295 | 시간 12[s] | 손실 3.10
| 에폭 1 |  반복 261 / 9295 | 시간 13[s] | 손실 3.00
| 에폭 1 |  반복 281 / 9295 | 시간 14[s] | 손실 2.95
| 에폭 1 |  반복 301 / 9295 | 시간 15[s] | 손실 2.91
| 에폭 1 |  반복 321 / 9295 | 시간 16[s] | 손실 2.85
| 에폭 1 |  반복 341 / 9295 | 시간 17[s] | 손실 2.82
| 에폭 1 |  반복 361 / 9295 | 시간 18[s] | 손실 2.83
| 에폭 1 |  반복 381 / 9295 | 시간 19[s] | 손실 2.80
| 에폭 1 |  반복 401 / 9295 | 시간 20[s] | 손실 2.76
| 에폭 1 |  반복 421 / 9295 | 시간 21[s] | 손실 2.73
| 에폭 1 |  반복 441 / 9295 | 